In [5]:
from collections import Counter
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# 🎯 Subsampling en Data Science: Oversampling vs Undersampling

Cuando trabajamos con datos en **machine learning**, es común encontrarnos con **datasets desbalanceados**, donde una categoría tiene muchas más muestras que otra. Por ejemplo, en un dataset de detección de fraude, las transacciones legítimas pueden representar el **99%** de los datos, mientras que las fraudulentas solo el **1%**.  

Este desbalance puede hacer que los modelos de machine learning sean **sesgados**, favoreciendo la clase mayoritaria y fallando en detectar la clase minoritaria. Para solucionar esto, podemos aplicar **subsampling**, que se divide en dos estrategias:

---

## 🔹 **1. Oversampling (Aumentar la clase minoritaria)**
### **¿Qué es?**
Consiste en **generar más ejemplos de la clase minoritaria** para balancear el dataset. Se puede hacer de dos maneras:
- **Duplicando ejemplos existentes.**
- **Generando nuevos datos sintéticos** (Ej. con el método SMOTE).

### **Ejemplo práctico** 👶🧑  
Imagina que tienes un dataset con **1000 hombres** y **100 mujeres**. Si aplicamos **oversampling**, podemos **aumentar el número de mujeres a 1000** duplicando datos o generando nuevos ejemplos sintéticos.

✅ **Ventajas**:
- Evita perder información de la clase mayoritaria.
- Funciona bien cuando hay pocos datos.

❌ **Desventajas**:
- Puede generar **overfitting** (si solo duplicamos datos).
- Si los datos generados no son representativos, el modelo puede aprender patrones incorrectos.

---

## 🔹 **2. Undersampling (Reducir la clase mayoritaria)**
### **¿Qué es?**
Consiste en **eliminar ejemplos de la clase mayoritaria** hasta balancear el dataset. En lugar de agregar más datos de la clase minoritaria, **quitamos datos de la clase dominante**.

### **Ejemplo práctico** 🚗🏍️  
Si en un dataset de accidentes de tráfico tenemos **5000 autos** y **200 motos**, podemos reducir los autos a **200** seleccionando una muestra representativa.

✅ **Ventajas**:
- Reduce el tiempo de entrenamiento.
- Puede ser útil cuando hay demasiados datos.

❌ **Desventajas**:
- Se pierde información importante.
- Puede generar un dataset no representativo.

---

## 🎯 **¿Cuándo usar cada método?**
| Situación | Método recomendado |
|-----------|------------------|
| Hay muy pocos datos | **Oversampling** |
| Dataset es muy grande y desbalanceado | **Undersampling** |
| Se quiere evitar pérdida de información | **Oversampling** |
| Se busca reducir el costo computacional | **Undersampling** |

🔹 **Si tienes pocos datos → usa oversampling.**  
🔹 **Si el dataset es enorme → usa undersampling.**  

A continuación, veremos cómo hacer esto en Python. 🚀


![Texto alternativo](https://www.researchgate.net/profile/Malak-Abdullah/publication/340978368/figure/fig1/AS:895469177810944@1590507897423/Differences-between-undersampling-and-oversampling.ppm)

https://www.researchgate.net/profile/Malak-Abdullah/publication/340978368/figure/fig1/AS:895469177810944@1590507897423/Differences-between-undersampling-and-oversampling.ppm

In [ ]:
# 📌 Simulación de datos desbalanceados
np.random.seed(42)
data = {
    'feature1': np.random.rand(1000),
    'feature2': np.random.rand(1000),
    'label': np.concatenate((np.zeros(900), np.ones(100)))  # 900 ejemplos de clase 0, 100 de clase 1
}

df = pd.DataFrame(data)
X, y = df[['feature1', 'feature2']], df['label']

print(f"Distribución original: {y.value_counts(1)}")

# 📊 1. Oversampling con SMOTE (Generar datos sintéticos para la clase minoritaria)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print(f"Después de Oversampling (SMOTE): {y_resampled.value_counts(1)}")

# 📊 2. Undersampling (Reducir la clase mayoritaria)
undersample = RandomUnderSampler(random_state=42)
X_under, y_under = undersample.fit_resample(X, y)
print(f"Después de Undersampling: {y_under.value_counts(1)}")


Distribución original: label
0.0    0.9
1.0    0.1
Name: proportion, dtype: float64
Después de Oversampling (SMOTE): label
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Después de Undersampling: label
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64


# 🎯 Undersampling en Data Science: Métodos Principales

Cuando trabajamos con **datasets desbalanceados**, una opción para equilibrar las clases es **undersampling**, que consiste en reducir la cantidad de datos de la clase mayoritaria.  

A continuación, exploraremos los **tres principales métodos de undersampling** y cómo funcionan.  

---

## 🔹 **1. Random Undersampling (Submuestreo Aleatorio)**
### **¿Cómo funciona?**
1. **Se identifican las clases en el dataset.**
2. **Se cuenta cuántos datos tiene cada clase.**
3. **Se eliminan muestras aleatorias de la clase mayoritaria** hasta que ambas clases tengan tamaños similares.

Este método es **rápido y simple**, pero puede eliminar datos importantes si los ejemplos eliminados contienen información valiosa.

![Texto alternativo](https://www.blog.trainindata.com/wp-content/uploads/2023/03/undersampling-1024x576.png)

### **Ejemplo práctico** 🎟️  
Imagina que tienes un dataset con:
- **900 transacciones legítimas** (clase mayoritaria).
- **100 transacciones fraudulentas** (clase minoritaria).

Con **random undersampling**, eliminamos **800 transacciones legítimas al azar**, quedándonos con **100 ejemplos de cada clase**.

✅ **Ventajas**:
- Método rápido y fácil de aplicar.
- Reduce el costo computacional.

❌ **Desventajas**:
- Puede eliminar datos importantes de la clase mayoritaria.
- Puede hacer que el modelo tenga menos información para aprender.

---

## 🔹 **2. Cluster Centroids (Reducción basada en centroides)**
### **¿Cómo funciona?**
1. **Agrupa los datos de la clase mayoritaria en clústeres** utilizando algoritmos como k-means.
2. **Calcula el centroide de cada clúster** (un punto representativo del grupo).
3. **Sustituye los datos originales por los centroides**, reduciendo la cantidad de ejemplos sin eliminarlos completamente.

Este método **preserva mejor la distribución de los datos**, ya que en lugar de eliminar puntos al azar, los reemplaza por valores representativos.

![Texto alternativo](https://www.researchgate.net/publication/373854510/figure/fig4/AS:11431281200027352@1697767140292/Clustering-based-under-sampling-majority-samples.png)


### **Ejemplo práctico** 📍  
Si tenemos **5000 registros de clientes no fraudulentos**, el algoritmo los agrupa en **200 clústeres** y los reemplaza por los centroides de cada grupo. Así, reducimos la cantidad de datos sin perder tanta información.

✅ **Ventajas**:
- Mantiene una representación más fiel de los datos originales.
- Es más preciso que eliminar datos al azar.

❌ **Desventajas**:
- Puede distorsionar las relaciones originales de los datos.
- Es más lento que el submuestreo aleatorio.

---

## 🔹 **3. NearMiss (Submuestreo basado en distancia)**
### **¿Cómo funciona?**
1. **Calcula la distancia entre cada punto de la clase mayoritaria y la clase minoritaria** usando métricas como la distancia euclidiana.
2. **Selecciona los puntos de la clase mayoritaria que estén más cerca de la clase minoritaria.**
3. **Elimina los puntos de la clase mayoritaria que están lejos**.

Este método se enfoca en **mantener los ejemplos de la clase mayoritaria que son más difíciles de distinguir de la clase minoritaria**.

Existen **tres variantes**:
- **NearMiss-1**: Selecciona los ejemplos más cercanos a la clase minoritaria.
- **NearMiss-2**: Selecciona los ejemplos más representativos de la clase mayoritaria.
- **NearMiss-3**: Usa una combinación de los dos anteriores.

![Texto alternativo](https://fritz.ai/wp-content/uploads/2023/09/18WM0gsh_naPEa9HTpE2c1A.webp)

### **Ejemplo práctico** 🏃‍♂️  
Si tenemos 1000 ejemplos de clase mayoritaria y 100 de clase minoritaria, **NearMiss** selecciona los **100 ejemplos más cercanos** a la clase minoritaria y elimina el resto.

✅ **Ventajas**:
- Mantiene ejemplos relevantes que pueden mejorar la capacidad del modelo para distinguir clases.
- Reduce el sesgo en la eliminación de datos.

❌ **Desventajas**:
- Es más lento que otros métodos.
- Si los datos no están bien distribuidos, puede eliminar patrones importantes.

Material extra:

https://towardsdatascience.com/oversampling-and-undersampling-explained-a-visual-guide-with-mini-2d-dataset-1155577d3091/

---

## 🎯 **¿Cuál método elegir?**
| Método | ¿Cuándo usarlo? |
|--------|----------------|
| **Random Undersampling** | Si quieres rapidez y simplicidad. |
| **Cluster Centroids** | Si quieres conservar representatividad de datos. |
| **NearMiss** | Si te interesa mantener ejemplos cercanos a la clase minoritaria. |

A continuación, implementaremos estos métodos en Python. 🚀


In [ ]:
from collections import Counter
import numpy as np
import pandas as pd


# 📌 Simulación de datos desbalanceados
np.random.seed(42)
data = {
    'feature1': np.random.rand(1000),
    'feature2': np.random.rand(1000),
    'label': np.concatenate((np.zeros(900), np.ones(100)))  # 900 ejemplos de clase 0, 100 de clase 1
}

df = pd.DataFrame(data)
X, y = df[['feature1', 'feature2']], df['label']

print(f"Distribución original: {y.value_counts(1)}")

# 📊 1. Random Undersampling (Eliminar ejemplos aleatorios de la clase mayoritaria)
undersample = RandomUnderSampler(sampling_strategy = 0.5,random_state=42)
X_under, y_under = undersample.fit_resample(X, y)
print(f"Después de Random Undersampling: {y_under.value_counts(1)}")

# 📊 2. Cluster Centroids (Reemplazar ejemplos con centroides de clústeres)
cluster_centroids = ClusterCentroids(sampling_strategy = 1, random_state=42)
X_cc, y_cc = cluster_centroids.fit_resample(X, y)
print(f"Después de Cluster Centroids: {y_cc.value_counts(1)}")

# 📊 3. NearMiss (Seleccionar ejemplos de la clase mayoritaria más cercanos a la clase minoritaria)
nearmiss = NearMiss(sampling_strategy = 0.2, version=1)
X_nm, y_nm = nearmiss.fit_resample(X, y)
print(f"Después de NearMiss: {y_nm.value_counts(1)}")


Distribución original: label
0.0    0.9
1.0    0.1
Name: proportion, dtype: float64
Después de Random Undersampling: label
0.0    0.666667
1.0    0.333333
Name: proportion, dtype: float64
Después de Cluster Centroids: label
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Después de NearMiss: label
0.0    0.833333
1.0    0.166667
Name: proportion, dtype: float64


# 🔄 **Oversampling en Data Science: Métodos Principales**

Cuando tenemos un **dataset desbalanceado**, una estrategia para equilibrar las clases es **oversampling**, que consiste en **aumentar la cantidad de datos de la clase minoritaria**.  

En esta sección exploraremos **tres métodos principales de oversampling**, explicando cómo funcionan y cuándo usarlos.  

---

## 🔹 **1. Random Oversampling (Sobremuestreo Aleatorio)**
### **¿Cómo funciona?**
1. **Se identifican las clases en el dataset.**
2. **Se cuenta cuántos datos tiene cada clase.**
3. **Se duplican ejemplos aleatorios de la clase minoritaria** hasta alcanzar un balance con la clase mayoritaria.

Este método **no genera datos nuevos**, simplemente **duplica** ejemplos ya existentes.

![Texto alternativo](https://www.researchgate.net/profile/Okan-Bulut-2/publication/367177472/figure/fig1/AS:11431281113412823@1673885738535/Random-oversampling-process.ppm)

### **Ejemplo práctico** 🎟️  
Imagina que tienes un dataset con:
- **900 transacciones legítimas** (clase mayoritaria).
- **100 transacciones fraudulentas** (clase minoritaria).

Con **random oversampling**, duplicamos **800 ejemplos de la clase minoritaria** hasta tener **900 en cada clase**.

✅ **Ventajas**:
- Es fácil de implementar.
- Funciona bien en datasets pequeños.

❌ **Desventajas**:
- Puede causar **overfitting**, ya que solo repite ejemplos sin agregar información nueva.
- No mejora la diversidad de los datos.

---

## 🔹 **2. SMOTE (Synthetic Minority Over-sampling Technique)**
### **¿Cómo funciona?**
1. **Selecciona un punto de la clase minoritaria**.
2. **Encuentra sus k vecinos más cercanos** (normalmente k=5).
3. **Crea nuevos puntos sintéticos** generando valores intermedios entre los puntos originales.

Este método **no duplica ejemplos**, sino que **genera nuevos puntos** mediante interpolación.

![Texto alternativo](https://www.researchgate.net/profile/Rohitash-Chandra/publication/359191299/figure/fig2/AS:1168122144927744@1655513430375/SMOTE-processing-for-oversampling.png)


✅ **Ventajas**:
- Evita overfitting porque no repite datos, sino que los sintetiza.
- Mejora la diversidad de la clase minoritaria.

❌ **Desventajas**:
- Puede generar datos poco realistas si la distribución de la clase minoritaria es muy dispersa.
- Requiere más tiempo de cómputo que el sobremuestreo aleatorio.

---

## 🔹 **3. ADASYN (Adaptive Synthetic Sampling)**
### **¿Cómo funciona?**
1. **Identifica los ejemplos de la clase minoritaria que están más aislados**.
2. **Genera más datos sintéticos en esas áreas** para compensar el desbalance.
3. **Usa un método similar a SMOTE**, pero con énfasis en los ejemplos difíciles de clasificar.

Este método es útil cuando la clase minoritaria **tiene una distribución compleja**, ya que se enfoca en **las regiones más difíciles de aprender**.

![Texto alternativo](https://towardsdatascience.com/wp-content/uploads/2021/03/10jwntVGaj7qQkr-MeueQcQ.jpeg)

### **Ejemplo práctico** 🔍  
Si tenemos datos minoritarios dispersos, **ADASYN genera más ejemplos en las zonas menos representadas**, ayudando a mejorar la precisión del modelo.

✅ **Ventajas**:
- Mejora la clasificación de los casos más difíciles.
- Se adapta mejor a distribuciones desbalanceadas.

❌ **Desventajas**:
- Puede generar ruido si se crean demasiados datos en áreas poco representativas.

---

## 🎯 **¿Cuál método elegir?**
| Método | ¿Cuándo usarlo? |
|--------|----------------|
| **Random Oversampling** | Si necesitas una solución rápida en datasets pequeños. |
| **SMOTE** | Si quieres mejorar la diversidad de los datos sin duplicar ejemplos. |
| **ADASYN** | Si hay zonas de la clase minoritaria que necesitan más ejemplos. |

A continuación, implementaremos estos métodos en Python. 🚀


In [ ]:
# 📌 Simulación de datos desbalanceados
np.random.seed(42)
data = {
    'feature1': np.random.rand(1000),
    'feature2': np.random.rand(1000),
    'label': np.concatenate((np.zeros(900), np.ones(100)))  # 900 ejemplos de clase 0, 100 de clase 1
}

df = pd.DataFrame(data)
X, y = df[['feature1', 'feature2']], df['label']

print(f"Distribución original: {y.value_counts(1)}")

# 🔄 1. Random Oversampling (Duplicar ejemplos de la clase minoritaria)
oversample = RandomOverSampler(random_state=42)
X_over, y_over = oversample.fit_resample(X, y)
print(f"Después de Random Oversampling: {y_over.value_counts(1)}")

# 🔄 2. SMOTE (Generar nuevos ejemplos sintéticos)
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)
print(f"Después de SMOTE: {y_smote.value_counts(1)}")

# 🔄 3. ADASYN (Generar datos en zonas difíciles de la clase minoritaria)
adasyn = ADASYN(random_state=42)
X_adasyn, y_adasyn = adasyn.fit_resample(X, y)
print(f"Después de ADASYN: {y_adasyn.value_counts(1)}")


Distribución original: label
0.0    0.9
1.0    0.1
Name: proportion, dtype: float64
Después de Random Oversampling: label
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Después de SMOTE: label
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Después de ADASYN: label
1.0    0.502212
0.0    0.497788
Name: proportion, dtype: float64


# Ejercicio

🏆 Predicción de Morosidad en Créditos  

El objetivo de este ejercicio es entrenar un modelo de **clasificación binario** para pronosticar las personas que **no pagaron el crédito** (`pago_credito = 1`).  
Dado que esta es la clase minoritaria en los datos, **se debe aplicar balanceo** antes de entrenar el modelo.  

## 📌 Pasos a Seguir  



 1️⃣ **Crear un pipeline de preprocesamiento en `sklearn`** para manejar los datos faltantes y transformar los datos.  
   - **Datos numéricos:** Imputar con la mediana y estandarizar.  
   - **Datos categóricos ordinales:** Imputar con el valor más frecuente y aplicar codificación ordinal.  
   - **Datos categóricos no ordinales:** Imputar con el valor más frecuente y aplicar One-Hot Encoding.  

2️⃣ **Aplicar técnicas de balanceo:**  
   - Primero, aplicar **undersampling** a la clase mayoritaria (`pago_credito = 0`).  
   - Luego, aplicar **oversampling** a la clase minoritaria (`pago_credito = 1`).  
   - El dataset final debe tener un balance de **60% (clase mayoritaria) vs 40% (clase minoritaria)**.

3️⃣ **Entrenar un modelo `ExtraTreesClassifier`** con los datos balanceados.  

4️⃣ **Elegir e implementar la métrica de evaluación más adecuada** para un problema de clasificación desbalanceada.  

¡Manos a la obra! 🚀  


In [6]:
# 📌 Fijar semilla para reproducibilidad
np.random.seed(42)

# 📌 Número de filas
n_filas = 5000

# 🔹 Columnas numéricas con valores faltantes
patrimonio_mensual = np.random.randint(500000, 50000000, size=n_filas).astype(float)
ingreso_mensual = np.random.randint(1000000, 10000000, size=n_filas).astype(float)
edad = np.random.randint(18, 70, size=n_filas).astype(float)

# 🔹 Columnas categóricas con valores faltantes
nivel_educativo_opciones = ["Primaria", "Secundaria", "Universitario", "Posgrado"]
tipo_empleo_opciones = ["Independiente", "Empleado", "Pensionado"]

nivel_educativo = np.random.choice(nivel_educativo_opciones, size=n_filas, p=[0.2, 0.3, 0.4, 0.1]).astype(object)
tipo_empleo = np.random.choice(tipo_empleo_opciones, size=n_filas, p=[0.3, 0.6, 0.1]).astype(object)

# 🔹 Target desbalanceado (95% no pagó, 5% sí pagó)
pago_credito = np.random.choice([0, 1], size=n_filas, p=[0.95, 0.05])

# 📌 Crear el DataFrame
df = pd.DataFrame({
    "patrimonio_mensual": patrimonio_mensual,
    "ingreso_mensual": ingreso_mensual,
    "edad": edad,
    "nivel_educativo": nivel_educativo,
    "tipo_empleo": tipo_empleo,
    "pago_credito": pago_credito
})

# 🔹 Introducir valores faltantes en todas las columnas excepto el target
for col in df.columns:
    if col != "pago_credito":
        mask = np.random.rand(n_filas) < 0.2  # 20% de valores nulos
        df.loc[mask, col] = np.nan

# 🔍 Visualizar primeras filas
df.head()

,patrimonio_mensual,ingreso_mensual,edad,nivel_educativo,tipo_empleo,pago_credito
0,NaN,9493535.0,29.0,Posgrado,Pensionado,0
1,13815092.0,8783874.0,30.0,NaN,Empleado,0
2,NaN,7590828.0,45.0,NaN,Independiente,0
3,NaN,5217690.0,NaN,Secundaria,NaN,0
4,48640618.0,4649886.0,68.0,Secundaria,Empleado,0


# Mi test

In [15]:
pd.set_option('display.float_format', '{:,.2f}'.format)

display(
    df.info(),
    df.describe(),
    df.isna().sum(),
    df.isna().mean(), # Sirve para ver porcentualmente mis datos ausentes
    df['pago_credito'].value_counts(1)
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patrimonio_mensual  4036 non-null   float64
 1   ingreso_mensual     3964 non-null   float64
 2   edad                4015 non-null   float64
 3   nivel_educativo     3991 non-null   object 
 4   tipo_empleo         4032 non-null   object 
 5   pago_credito        5000 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 234.5+ KB


None

,patrimonio_mensual,ingreso_mensual,edad,pago_credito
count,"4,036.00","3,964.00","4,015.00","5,000.00"
mean,"25,496,402.31","5,471,614.55",43.55,0.05
std,"14,327,458.09","2,602,829.51",14.88,0.22
min,"504,138.00","1,000,037.00",18.00,0.00
25%,"12,923,640.50","3,163,573.75",31.00,0.00
50%,"25,473,303.00","5,406,506.50",43.00,0.00
75%,"38,060,951.75","7,735,836.00",57.00,0.00
max,"49,986,268.00","9,998,331.00",69.00,1.00


patrimonio_mensual     964
ingreso_mensual       1036
edad                   985
nivel_educativo       1009
tipo_empleo            968
pago_credito             0
dtype: int64

patrimonio_mensual   0.19
ingreso_mensual      0.21
edad                 0.20
nivel_educativo      0.20
tipo_empleo          0.19
pago_credito         0.00
dtype: float64

pago_credito
0   0.95
1   0.05
Name: proportion, dtype: float64

In [14]:
display(
    df['tipo_empleo'].unique(),
    df['nivel_educativo'].unique()
)

array(['Pensionado', 'Empleado', 'Independiente', nan], dtype=object)

array(['Posgrado', nan, 'Secundaria', 'Universitario', 'Primaria'],
      dtype=object)

In [17]:
# 📌 Separar características y target
X = df.drop(columns=["pago_credito"])
y = df["pago_credito"]

In [18]:
# 🔹 Identificar columnas por tipo
num_features = ["patrimonio_mensual", "ingreso_mensual", "edad"]
ord_features = ["nivel_educativo"]
cat_features = ["tipo_empleo"]

# 🔹 Pipelines de preprocesamiento
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

ord_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(categories=[["Primaria", "Secundaria", "Universitario", "Posgrado"]]))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# 🔹 Combinar en un preprocesador general
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("ord", ord_pipeline, ord_features),
    ("cat", cat_pipeline, cat_features)
])

transformed_data = preprocessor.fit_transform(X)


cat_ord_clean_cols = (
    preprocessor
    .named_transformers_
    ['cat']
    ['encoder']
    .get_feature_names_out(cat_features)
    .tolist()
)

data_clean = pd.DataFrame(
    transformed_data,
    columns = num_features+ord_features+cat_ord_clean_cols
)

In [19]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['patrimonio_mensual', 'ingreso_mensual',
                                  'edad']),
                                ('ord',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OrdinalEncoder(categories=[['Primaria',
                                                                              'Secundaria',
                                                                              'Universitario',
                                                                              'Posgrado']]))]),
                                 ['nivel_educativo']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['tipo_empleo'])])

In [ ]:
# 📌 Balanceo de datos: Aplicar Undersampling y luego Oversampling
undersample = RandomUnderSampler(sampling_strategy=0.3, random_state=42)  # Reducir clase mayoritaria
X_under, y_under = undersample.fit_resample(data_clean, y)

oversample = SMOTE(sampling_strategy=0.67, random_state=42)  # Aumentar clase minoritaria
X_resampled, y_resampled = oversample.fit_resample(X_under, y_under)


In [22]:
y_resampled.value_counts(1)

pago_credito
0   0.60
1   0.40
Name: proportion, dtype: float64

In [23]:
# 📌 División en train y test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 📌 Modelo ExtraTrees
model = ExtraTreesClassifier(n_estimators=100, random_state=42)

# 🔥 Entrenar el modelo
model.fit(X_train, y_train)

# 📌 Predicciones y evaluación
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.85      0.79       263
           1       0.72      0.57      0.64       180

    accuracy                           0.74       443
   macro avg       0.73      0.71      0.71       443
weighted avg       0.73      0.74      0.73       443



# Solucion

In [ ]:
# 📌 Separar características y target
X = df.drop(columns=["pago_credito"])
y = df["pago_credito"]

In [ ]:
# 🔹 Identificar columnas por tipo
num_features = ["patrimonio_mensual", "ingreso_mensual", "edad"]
ord_features = ["nivel_educativo"]
cat_features = ["tipo_empleo"]

# 🔹 Pipelines de preprocesamiento
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

ord_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(categories=[["Primaria", "Secundaria", "Universitario", "Posgrado"]]))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# 🔹 Combinar en un preprocesador general
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("ord", ord_pipeline, ord_features),
    ("cat", cat_pipeline, cat_features)
])

transformed_data = preprocessor.fit_transform(X)


cat_ord_clean_cols = (
    preprocessor
    .named_transformers_
    ['cat']
    ['encoder']
    .get_feature_names_out(cat_features)
    .tolist()
)

data_clean = pd.DataFrame(
    transformed_data,
    columns = num_features+ord_features+cat_ord_clean_cols
)

,patrimonio_mensual,ingreso_mensual,edad,nivel_educativo,tipo_empleo_Empleado,tipo_empleo_Independiente,tipo_empleo_Pensionado
0,-0.001449,1.741353,-1.083036,3.0,0.0,0.0,1.0
1,-0.907235,1.435121,-1.008026,2.0,1.0,0.0,0.0
2,-0.001449,0.920300,0.117120,2.0,0.0,1.0,0.0
3,-0.001449,-0.103752,-0.032899,1.0,1.0,0.0,0.0
4,1.798540,-0.348769,1.842345,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
4995,-1.870470,1.517819,-0.032899,2.0,0.0,1.0,0.0
4996,-0.702778,-0.022274,-0.332938,1.0,0.0,1.0,0.0
4997,-0.001449,1.660226,0.567179,0.0,1.0,0.0,0.0
4998,-1.552016,0.933174,-1.158046,2.0,0.0,0.0,1.0


In [20]:
# 📌 Balanceo de datos: Aplicar Undersampling y luego Oversampling
undersample = RandomUnderSampler(sampling_strategy=0.3, random_state=42)  # Reducir clase mayoritaria
X_under, y_under = undersample.fit_resample(data_clean, y)

oversample = SMOTE(sampling_strategy=0.67, random_state=42)  # Aumentar clase minoritaria
X_resampled, y_resampled = oversample.fit_resample(X_under, y_under)

In [ ]:
# 📌 División en train y test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# 📌 Modelo ExtraTrees
model = ExtraTreesClassifier(n_estimators=100, random_state=42)

# 🔥 Entrenar el modelo
model.fit(X_train, y_train)

# 📌 Predicciones y evaluación
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.86      0.80       263
           1       0.73      0.57      0.64       180

    accuracy                           0.74       443
   macro avg       0.74      0.71      0.72       443
weighted avg       0.74      0.74      0.73       443

